In [3]:
from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("summarization")

# Text to summarize
text = """
The Viterbi algorithm is a dynamic programming algorithm used for finding the most likely sequence of hidden states—called the Viterbi path—that results in a sequence of observed events, especially in the context of Markov information sources and hidden Markov models (HMM). It was introduced by Andrew Viterbi in 1967 for decoding convolutional codes over noisy digital communication links. Since then, it has found widespread use in various areas such as digital communications, speech recognition, bioinformatics, and natural language processing.

The algorithm operates by constructing a trellis (a graph that organizes the possible states in a temporal sequence) for the sequence of observed events and then finding the path through this trellis that has the highest probability, taking into account both the probabilities of transitions between states and the probabilities of observations given the states. This path represents the sequence of states that most likely explains the sequence of observations.

The key steps of the Viterbi algorithm include:

Initialization: At the first time step, the initial probabilities of being in each state are used to initialize the algorithm.

Recursion: For each subsequent time step, the algorithm calculates the probability of each state being the end of a path from the start state through the previous time step and then entering the current state. It keeps track of the path that has the highest probability at each step.

Termination: At the final time step, the algorithm identifies the state with the highest probability as the end of the most likely path.

Path Backtracking: Starting from the identified end state, the algorithm traces back through the trellis using pointers that were stored during the recursion step to find the sequence of states that constitutes the Viterbi path.

The Viterbi algorithm is efficient, requiring time and space that are both linear in the number of time steps and the number of states, making it practical for a wide range of applications.
"""

# Generate summary
summary = summarizer(text, max_length=200, min_length=20, do_sample=False)

# Print the summarized text
print(summary[0]['summary_text'])


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 The Viterbi algorithm is a dynamic programming algorithm used for finding the most likely sequence of hidden states . It was introduced by Andrew Vitbi in 1967 for decoding convolutional codes over noisy digital communication links . The algorithm operates by constructing a trellis (a graph that organizes the possible states in a temporal sequence) for the sequence of observed events .


In [8]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

# Load pre-trained model tokenizer (vocabulary) and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

# Choose the device (CPU or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

text = "Data Science is"
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)

# Set the maximum length of the sequence to generate
max_length = 50

# Generate text using top-p sampling
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=max_length,
    temperature=1.0, # Controls randomness: lower -> less random
    top_p=0.92, # Nucleus sampling
    top_k=0, # Disable top-k sampling
    repetition_penalty=1.2, # Penalize repetition
    do_sample=True, # Enable sampling to introduce randomness
    num_return_sequences=1, # Number of sequences to return
)

# Decode the generated sequence to a readable string
generated_sequence = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

print(generated_sequence)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Data Science is investigating South Africa's traffic safety scandal and designating the area Nesotia, founded in 1900 by Professor John "F" Munk.
The workshop opened on 18 April 2010 at Complexity Floor 3690 Chengwerui Road


BERT (Bidirectional Encoder Representations from Transformers) is a revolutionary method in the field of natural language processing (NLP) that has been pre-trained on a large corpus of text and can be fine-tuned for various NLP tasks. Here are a few small examples to illustrate the application of BERT in different scenarios:

### Sentiment Analysis
Sentiment analysis involves determining whether a piece of text is positive, negative, or neutral. BERT can be fine-tuned on a dataset of product reviews or social media posts to classify the sentiment of the text.

```python
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn import functional as F
import torch

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Example text
text = "The product was great and met all my expectations."

# Tokenize input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Predict sentiment
with torch.no_grad():
    logits = model(**inputs).logits
    probabilities = F.softmax(logits, dim=1)
    sentiment = torch.argmax(probabilities, dim=1)

# sentiment could be 0 (negative), 1 (neutral), or 2 (positive)
print(f"Sentiment: {sentiment.item()}")
```

### Question Answering
BERT can be used to build a question-answering system where the model finds an answer to a question from a given text.

```python
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# Load pre-trained model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Example text and question
text = "BERT stands for Bidirectional Encoder Representations from Transformers."
question = "What does BERT stand for?"

# Encode text
inputs = tokenizer.encode_plus(question, text, return_tensors='pt')
input_ids = inputs["input_ids"].tolist()[0]

# Get model predictions
answers_start_scores, answers_end_scores = model(**inputs, return_dict=False)
answer_start = torch.argmax(answers_start_scores)
answer_end = torch.argmax(answers_end_scores) + 1

# Convert answer back to tokens
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

print(f"Answer: {answer}")
```

### Text Classification
Text classification involves categorizing text into predefined categories. BERT can be fine-tuned on news articles, emails, or any text data to classify them into categories like "Sports," "Politics," "Technology," etc.

```python
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load model and tokenizer
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)  # Assuming 4 categories
tokenizer = BertTokenizer.from_pretrained(model_name)

# Example text
text = "The new technology innovation has been a game-changer in the industry."

# Tokenize and predict
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
output = model(**inputs)

# Assuming the labels are 0=Technology, 1=Politics, 2=Sports, 3=Entertainment
label = torch.argmax(output.logits, dim=1).item()

print(f"Category: {label}")
```

These examples showcase the versatility of BERT in handling various NLP tasks, demonstrating its power in understanding and generating insights from text data.

In [9]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# Load pre-trained model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Example text and question
text = "BERT stands for Bidirectional Encoder Representations from Transformers."
question = "What does BERT stand for?"

# Encode text
inputs = tokenizer.encode_plus(question, text, return_tensors='pt')
input_ids = inputs["input_ids"].tolist()[0]

# Get model predictions
answers_start_scores, answers_end_scores = model(**inputs, return_dict=False)
answer_start = torch.argmax(answers_start_scores)
answer_end = torch.argmax(answers_end_scores) + 1

# Convert answer back to tokens
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

print(f"Answer: {answer}")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Answer: bidirectional encoder representations from transformers
